# 🧪 Environment Check

Run this to verify Python, PyTorch, CUDA, and your GPU are working on this pod.

In [ ]:
import platform, shutil, subprocess
import torch

print("Python version:", platform.python_version())
print("PyTorch version:", torch.__version__)
print("CUDA version   :", torch.version.cuda)
print("CUDA available :", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU name       :", torch.cuda.get_device_name(0))
    props = torch.cuda.get_device_properties(0)
    print("GPU memory (GB):", round(props.total_memory / (1024**3), 2))

if shutil.which("nvidia-smi"):
    print("\n# nvidia-smi:\n")
    print(subprocess.check_output(["nvidia-smi"]).decode())
else:
    print("\n(nvidia-smi not found in PATH — that's OK on some hosts.)")


In [ ]:
# Simple GPU test
if torch.cuda.is_available():
    x = torch.randn(1_000, 1_000, device="cuda")
    y = torch.mm(x, x)
    print("GPU matmul OK; result mean:", y.mean().item())
else:
    print("CUDA not available; skipping GPU tensor test.")
